In [ ]:
from youtubeAPI import get_youtube_api_json
from pymongo import MongoClient # you should install dnspython also
from utils import ProgressBar
from youtubesearchpython import *
from pprint import pprint

with open("apikey.txt", encoding="utf8") as f:
    apikey1 = f.readline().strip()
    apikey2 = f.readline().strip()

with open("mongodb.txt") as f:
    mongodb_uri = f.readline().strip()

client = MongoClient(mongodb_uri)
db = client["youtube"]
searchData = db["searchData"]

In [ ]:
videosSearch = VideosSearch("")
pprint(videosSearch.result())

In [ ]:
suggestionData = db["suggestionData"]
suggestions = Suggestions(language="kr", region="KR")

In [ ]:
import re

hangul = re.compile("[^가-힣]+")

In [ ]:
total_data = set()
# for i in range(44032, 55216):
start, end = 54000, 55216
error_count = 0
bar = ProgressBar(end - start)
for i in range(start, end):
    bar.next()
    total_data.add(chr(i))
    try:
        data = suggestions.get(chr(i), mode=ResultMode.dict)
        for j in data["result"]:
            if len(hangul.findall(j)) == 0:
                total_data.add(j)
    except:
        error_count += 1
total_data = list(total_data)
total_data.sort()
print(error_count, total_data)

In [ ]:
suggestionData.insert_one({"result": total_data})

In [ ]:
all_data = list(suggestionData.find())
finalSuggestionData = db["finalSuggestionData"]
total_search_data = set()
for data in all_data:
    for keyword in data["result"]:
        total_search_data.add(keyword)
total_search_data = list(total_search_data)
total_search_data.sort()
print(len(total_search_data))

In [ ]:
finalSuggestionData.drop()

In [ ]:
divide = 50
size = len(total_search_data) // divide + 1
for i in range(divide):
    finalSuggestionData.insert_one(
        {"id": i, "result": total_search_data[size * i : min(size * (i + 1), len(total_search_data))]}
    )

In [ ]:
Video.get("MhQKe-aERsU")

In [ ]:
finalSuggestionData = db["finalSuggestionData"]

for data_num in range(37, 50):
    # data_num = 8
    total = finalSuggestionData.find_one({"id": data_num})["result"]
    id_collection = set()
    bar = ProgressBar(len(total))
    error_count = 0
    for target in total:
        bar.next()
        customSearch = CustomSearch(target, "CAMSAhAB", language="kr", region="KR")
        while True:
            search_result = customSearch.result()["result"]
            if len(search_result) == 0:
                break
            for result in search_result:
                try:
                    viewCount = int(result["viewCount"]["text"].split(" views")[0].replace(",", ""))
                    if viewCount < 10**6:
                        break
                    id_collection.add(result["id"])
                except:
                    continue
            else:
                customSearch.next()
                continue
            break
    print("count:",data_num, error_count, len(id_collection))
    videoIdData = db["videoIdData"]
    videoIdData.insert_one({"id":data_num, "result": list(id_collection)})

In [ ]:
print(list(id_collection))

In [ ]:
duplicate_datas = list(videoIdData.find())
remove_duplicate = set()
for duplicate_data in duplicate_datas["result"]:
    for id in duplicate_data:
        remove_duplicate.add(id)
remove_duplicate = list(remove_duplicate)
finalVideoIdData = db["finalVideoIdData"]
divide = 50
size = len(remove_duplicate) // divide + 1
for i in range(divide):
    finalVideoIdData.insert_one(
        {"id": i, "result": remove_duplicate[size * i : min(size * (i + 1), len(remove_duplicate))]}
    )

In [ ]:
global_query = {"key": apikey1, "maxResults": 100, "regionCode": "KR"}


def get_video(query):
    final_query = {
        **global_query,
        **query,
        "part": "snippet, contentDetails, status, statistics, player, topicDetails, recordingDetails",
    }
    return get_youtube_api_json("videos", final_query)

In [ ]:
videoInfoData = db["videoInfoData"]
final_total_id = list(finalVideoIdData.find())
for ids in final_total_id["result"]:
    count = 0
    result = []
    while len(ids) > 50 * count:
        query = {"id": ",".join(ids[50 * count : min(50 * (count + 1), len(ids))])}
        result.extend(get_video(query).get("items", []))
        count += 1
    videoInfoData.insert_one({"result": result})

In [ ]:
# preprocessing
import datetime
import time

def strftime(datetime: datetime.datetime):
    return datetime.strftime("%Y-%m-%dT%H:%M:%SZ")

# 1970-01-01T00:00:00Z
def strptime(str):
    return datetime.datetime().strptime(str, "%Y-%m-%dT%H:%M:%SZ")

# P1DT6H47M23S
def strpduration(str):
    return time.strptime()

In [ ]:
finalVideoInfoData = db["finalVideoInfoData"]
init_data = list(videoInfoData.find())
preprocessed_data = []
for data in init_data["result"]:
    preprocessed_data.append(
        {
            "id": data["id"],
            "publishedAt": data["snippet"].get("publishedAt"),
            "channelId": data["snippet"].get("channelId"),
            "title": data["snippet"].get("title"),
            "description": data["snippet"].get("description"),
            "channelTitle":data["snippet"].get("channelTitle"),
            "tags":data["snippet"].get("tags"),
            "categoryId":data["snippet"].get("categoryId"),
            "duration":data["contentDetails"].get("duration"),
            "publicStatsViewable":data["status"].get("publicStatsViewable"),
            "madeForKids":data["status"].get("madeForKids"),
            "viewCount":data["statistics"].get("viewCount"),
            "likeCount":data["statistics"].get("likeCount"),
            "favoriteCount":data["statistics"].get("favoriteCount")
        }
    )

In [ ]:
customSearch12 = CustomSearch("애플", "CAMSAhAB", language="kr", region="KR")  # preprocess english or not?
customSearch12.result()